# Explainable AI

## Image Recognition models with What-If tool

In [1]:
from IPython.display import Markdown, display

display(Markdown("./md/WIT_Smile_Detector.md"))

### What-If Tool Image Smile Detection

In this demo we demonstrate the use of what-if-tool for image recognition models. Our task is to predict if a person is smiling or not. We provide a CNN that is trained on a subset of [CelebA dataset](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) and visualize the results on a separate test subset.

Copyright 2019 Google LLC.
SPDX-License-Identifier: Apache-2.0



```python
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.1
```

#### **Re-run** the below cell until you see the output `Successfully installed h5py-2.10.0`.


```python
# Install version 2.10 of h5py
import sys
!{sys.executable} -m pip uninstall -y h5py
!{sys.executable} -m pip install 'h5py < 3.0.0'
```

    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
    Collecting h5py<3.0.0
      Downloading h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
    [K     |████████████████████████████████| 2.9 MB 5.0 MB/s eta 0:00:01
    [?25hRequirement already satisfied: numpy>=1.7 in /opt/conda/lib/python3.7/site-packages (from h5py<3.0.0) (1.19.5)
    Requirement already satisfied: six in /opt/conda/lib/python3.7/site-packages (from h5py<3.0.0) (1.16.0)
    Installing collected packages: h5py
    [31mERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
    tensorflow 2.1.4 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.19.5 which is incompatible.
    tensorflow 2.1.4 requires tensorboard<2.2.0,>=2.1.0, but you have tensorboard 2.6.0 which is incompatible.
    tensorflow-model-analysis 0.21.6 requires pyarrow<1,>=0.15, but you have pyarrow 5.0.0 which is incompatible.
    tensorflow-model-analysis 0.21.6 requires scipy==1.4.1; python_version >= "3", but you have scipy 1.7.1 which is incompatible.
    tensorflow-io 0.11.0 requires tensorflow==2.1.0, but you have tensorflow 2.1.4 which is incompatible.
    tensorflow-data-validation 0.21.5 requires joblib<0.15,>=0.12, but you have joblib 1.0.1 which is incompatible.
    tensorflow-data-validation 0.21.5 requires pandas<1,>=0.24, but you have pandas 1.3.2 which is incompatible.
    tensorflow-data-validation 0.21.5 requires scikit-learn<0.22,>=0.18, but you have scikit-learn 0.24.2 which is incompatible.[0m
    Successfully installed h5py-2.10.0


#### **Note**: Please ignore any **incompatibility ERROR** that may appear for the packages visions as it will not affect the lab's functionality.

### In order to use the correct __h5py__ version, you will need to restart the notebook's kernel. To do this, select __Kernel__ > __Restart Kernel__ from the top menu.

#### Download the pretrained keras model files and subset of celeba images


```python
!curl -L https://storage.googleapis.com/what-if-tool-resources/smile-demo/smile-colab-model.hdf5 -o ./smile-model.hdf5
!curl -L https://storage.googleapis.com/what-if-tool-resources/smile-demo/test_subset.zip -o ./test_subset.zip
!unzip -qq -o test_subset.zip
```

      % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                     Dload  Upload   Total   Spent    Left  Speed
    100 39.6M  100 39.6M    0     0   148M      0 --:--:-- --:--:-- --:--:--  148M
      % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                     Dload  Upload   Total   Spent    Left  Speed
    100 3819k  100 3819k    0     0   149M      0 --:--:-- --:--:-- --:--:--  149M


#### Define helper functions for dataset conversion from csv to tf.Examples


```python
import numpy as np
import tensorflow as tf
import os
from PIL import Image
from io import BytesIO

# Converts a dataframe into a list of tf.Example protos.
# If images_path is specified, it assumes that the dataframe has a special 
# column "image_id" and the path "images_path/image_id" points to an image file.
# Given this structure, this function loads and processes the images as png byte_lists
# into tf.Examples so that they can be shown in WIT. Note that 'image/encoded'
# is a reserved field in WIT for encoded image features.
def df_to_examples(df, columns=None, images_path=''):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if df[col].dtype is np.dtype(np.int64):
                example.features.feature[col].int64_list.value.append(int(row[col]))
            elif df[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            elif row[col] == row[col]:
                example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
        if images_path:
            fname = row['image_id']
            with open(os.path.join(images_path, fname), 'rb') as f:
                im = Image.open(f)
                buf = BytesIO()
                im.save(buf, format= 'PNG')
                im_bytes = buf.getvalue()
                example.features.feature['image/encoded'].bytes_list.value.append(im_bytes)
        examples.append(example)
    return examples

# Converts a dataframe column into a column of 0's and 1's based on the provided test.
# Used to force label columns to be numeric for binary classification using a TF estimator.
def make_label_column_numeric(df, label_column, test):
    df[label_column] = np.where(test(df[label_column]), 1, 0)
```

#### Load the csv file into pandas dataframe and process it for WIT


```python
import pandas as pd

data = pd.read_csv('celeba/data_test_subset.csv')
examples = df_to_examples(data, images_path='celeba/img_test_subset_resized/')
```


```python
data.info()
```

    <class 'pandas.core.frame.DataFrame'>
    RangeIndex: 2000 entries, 0 to 1999
    Data columns (total 42 columns):
     #   Column               Non-Null Count  Dtype 
    ---  ------               --------------  ----- 
     0   Unnamed: 0           2000 non-null   int64 
     1   image_id             2000 non-null   object
     2   5_o_Clock_Shadow     2000 non-null   int64 
     3   Arched_Eyebrows      2000 non-null   int64 
     4   Attractive           2000 non-null   int64 
     5   Bags_Under_Eyes      2000 non-null   int64 
     6   Bald                 2000 non-null   int64 
     7   Bangs                2000 non-null   int64 
     8   Big_Lips             2000 non-null   int64 
     9   Big_Nose             2000 non-null   int64 
     10  Black_Hair           2000 non-null   int64 
     11  Blond_Hair           2000 non-null   int64 
     12  Blurry               2000 non-null   int64 
     13  Brown_Hair           2000 non-null   int64 
     14  Bushy_Eyebrows       2000 non-null   int64 
     15  Chubby               2000 non-null   int64 
     16  Double_Chin          2000 non-null   int64 
     17  Eyeglasses           2000 non-null   int64 
     18  Goatee               2000 non-null   int64 
     19  Gray_Hair            2000 non-null   int64 
     20  Heavy_Makeup         2000 non-null   int64 
     21  High_Cheekbones      2000 non-null   int64 
     22  Male                 2000 non-null   int64 
     23  Mouth_Slightly_Open  2000 non-null   int64 
     24  Mustache             2000 non-null   int64 
     25  Narrow_Eyes          2000 non-null   int64 
     26  No_Beard             2000 non-null   int64 
     27  Oval_Face            2000 non-null   int64 
     28  Pale_Skin            2000 non-null   int64 
     29  Pointy_Nose          2000 non-null   int64 
     30  Receding_Hairline    2000 non-null   int64 
     31  Rosy_Cheeks          2000 non-null   int64 
     32  Sideburns            2000 non-null   int64 
     33  Smiling              2000 non-null   int64 
     34  Straight_Hair        2000 non-null   int64 
     35  Wavy_Hair            2000 non-null   int64 
     36  Wearing_Earrings     2000 non-null   int64 
     37  Wearing_Hat          2000 non-null   int64 
     38  Wearing_Lipstick     2000 non-null   int64 
     39  Wearing_Necklace     2000 non-null   int64 
     40  Wearing_Necktie      2000 non-null   int64 
     41  Young                2000 non-null   int64 
    dtypes: int64(41), object(1)
    memory usage: 656.4+ KB



```python
data.head()
```




<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Unnamed: 0</th>
      <th>image_id</th>
      <th>5_o_Clock_Shadow</th>
      <th>Arched_Eyebrows</th>
      <th>Attractive</th>
      <th>Bags_Under_Eyes</th>
      <th>Bald</th>
      <th>Bangs</th>
      <th>Big_Lips</th>
      <th>Big_Nose</th>
      <th>...</th>
      <th>Sideburns</th>
      <th>Smiling</th>
      <th>Straight_Hair</th>
      <th>Wavy_Hair</th>
      <th>Wearing_Earrings</th>
      <th>Wearing_Hat</th>
      <th>Wearing_Lipstick</th>
      <th>Wearing_Necklace</th>
      <th>Wearing_Necktie</th>
      <th>Young</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>193352</td>
      <td>193353.jpg</td>
      <td>0</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>...</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>179706</td>
      <td>179707.jpg</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>...</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>80689</td>
      <td>080690.jpg</td>
      <td>0</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>...</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
    </tr>
    <tr>
      <th>3</th>
      <td>40324</td>
      <td>040325.jpg</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>1</td>
      <td>...</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>182866</td>
      <td>182867.jpg</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>...</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
    </tr>
  </tbody>
</table>
<p>5 rows × 42 columns</p>
</div>




```python
examples[0]
```




    features {
      feature {
        key: "5_o_Clock_Shadow"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Arched_Eyebrows"
        value {
          int64_list {
            value: 1
          }
        }
      }
      feature {
        key: "Attractive"
        value {
          int64_list {
            value: 1
          }
        }
      }
      feature {
        key: "Bags_Under_Eyes"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Bald"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Bangs"
        value {
          int64_list {
            value: 1
          }
        }
      }
      feature {
        key: "Big_Lips"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Big_Nose"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Black_Hair"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Blond_Hair"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Blurry"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Brown_Hair"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Bushy_Eyebrows"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Chubby"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Double_Chin"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Eyeglasses"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Goatee"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Gray_Hair"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Heavy_Makeup"
        value {
          int64_list {
            value: 1
          }
        }
      }
      feature {
        key: "High_Cheekbones"
        value {
          int64_list {
            value: 1
          }
        }
      }
      feature {
        key: "Male"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Mouth_Slightly_Open"
        value {
          int64_list {
            value: 1
          }
        }
      }
      feature {
        key: "Mustache"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Narrow_Eyes"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "No_Beard"
        value {
          int64_list {
            value: 1
          }
        }
      }
      feature {
        key: "Oval_Face"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Pale_Skin"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Pointy_Nose"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Receding_Hairline"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Rosy_Cheeks"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Sideburns"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Smiling"
        value {
          int64_list {
            value: 1
          }
        }
      }
      feature {
        key: "Straight_Hair"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Unnamed: 0"
        value {
          int64_list {
            value: 193352
          }
        }
      }
      feature {
        key: "Wavy_Hair"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Wearing_Earrings"
        value {
          int64_list {
            value: 1
          }
        }
      }
      feature {
        key: "Wearing_Hat"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Wearing_Lipstick"
        value {
          int64_list {
            value: 1
          }
        }
      }
      feature {
        key: "Wearing_Necklace"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Wearing_Necktie"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "Young"
        value {
          int64_list {
            value: 0
          }
        }
      }
      feature {
        key: "image/encoded"
        value {
          bytes_list {
            value: "\211PNG\r\n\032\n\000\000\000\rIHDR\000\000\000@\000\000\000N\010\002\000\000\000\037\001\207\371\000\000%\254IDATx\234\225\273\311\257fG\226\037v\316\211\341\016\337\374\27593\231\231\314L2\223d\261\252X\335\354\352\252\352\352r[\255v\267\032\026\004h%HZ5 \303\200\ry\243\255\2016\340\255\377\004{\241\205\026\332\t\232\334\206\033*T7\324\223X\023\213d\026\223L&3\337\374\3367\336)\246s\274\370^&\263JE\331\216\305\367\342]\334\033q\2468\021\347\234_\340\017\037\036\002\000\"\"\"\000\000\200\210\000\0003\303\257j:u\000\000@\002\300H\002\010\000\202\220\004\225R\306\030\233ic\014\021m\306\214\334\305\310\nI)\345\273\256\252*Mf0\030\000\"\0000\" 2\002\020\n\020\000\200\3518!$\022\326 F\241V\n\025\211\322\222B\207\230\2001\204$Lm\345NO\316\364\206\334M\333\360\200\210\"\362\202\237_j\210\n\000\004\000\2016\257\003\000 \010\313\2461\363\206\371\253\021\220\020\277\030\215\210\210\350\305`_\214+\004\277bB\006`\000\365B\304\210x\325#BD\000\322\314\374\262\370_\360\360e\032@\304\r\365\214\317i\277\372\371B\020\033N\256\372 \000\300 \260\341\n\024 \262\210\"\265\031A\000\020\225<\037\034\005@@6\263\274D\030s\024I\210W\203#P\022\360)\352_\"\356eQ\375j\006D#\000\343F\337\360\302\204\254\006\324\244\r)\245\224R/>O\222\276\370\026\224\326\232\210\020\351\305T\210*! \000\022nXBD\000$\"\274\222\255\000p\214\021\230E\204\031\230\205PRJ\336GM\200\010H\200\204\204\200\370B\221_bB@\n\000^0\'\317\337\322V+\245\264\326d\364\313\014\340F`\210(D\022\021\221\210\2141Wz\023\002B@\020\000 $\000\2712\022\002!T\352\205\311\200\300\306pD\004\t\025\031D\225R\322J\251\347FE/\333\375\227\255\001AyN\331\027\177\004Ak\245\224RZ)\"E$\204\033\303B\324\033!\212\240\000j\255\025\021\241\212\234\024\"\003\t\002\022\n\200\340f\371\221\020\201\020(\005\033\341^\tX\001\242F@dB\324\312h\223!i\375\234\376\r\301//\350_\255\200\315\222\002`\274\242~cP@ (L\302\010\010\210\310\304\270\221\275\\\t\230\023\000*\"D\024`MJ6<\242b\334\254\004\024\004f\272\022\377\306\232\004pcUHH\202D\304\200\240\265\006\"\r@\372\205\340\177\311\215~\231\006\236\273\016D\024\332\274\t\002\300\230\230@\0013\260\205(\202\004\010 \022!)@\000\222\304\"\002H\210(\202\326ZA $F\241\215\'\005\021\270\342\t\2216\003#]\351\231%m&\336(\215\001\030(2j\272\222\317\027N\2146\037\t\274\314\330\206+\021\211)\001\200&4J\021\002J\"\"M\346rv\036c\004\304\242?\350\017\206d\264\353B\323\265H\"\"1p\n\021E\264\326\231-\254\265&\317\2420\210\020b\010\301\307@J\233<\013)\031\243\tUJ@\004ZCb\350:\227\345F$y\357\007\375^S\203\367\240\224>:<\321\360\205\223\376\262\316F! \"\210\300\201\255V(\354\235\313-\021\342\321\323\317\036\177\362\361r\271L)\201\242\376`\264\275\263\267\265\2737\030\016\255\315Vuu~~\336Vu\231\227eYr\214\3169\020\032\215F\327^\2711\236N:\037\021x4\350\tB\333v\275\242 \324\314\010\2119I\020\"\005y\236\013\004\" \242\310\220\004\004\241s\241jZ\375\237o\002\277\262\363\242Y\255\224&%\000\204\241\353\016O\236\375\364\'\357=\374\340\203\253\315K\253\274(G\323\255\275\375k\373\007\327\007\343Q\3234\227\227\227\276\355\326J5u}tx\342\272\256\327\353!\321\366\366\316k\367_\177\375\301\203\361t+\304\030S\314\024\301\325\026\244\224\"\000H)\t\221\315\320\271\244\224&\215!\260\010!@]\267\313\305J+\300\027\326\362r\347\212zy\331\204PD\264\321\336wV+$zv\364\371_\374\331\367\037}\374Q\364N\241\350\314Z\316\235\244s\327U\313\305\331\351q\257\337\267\326\266m\353\234k\353\356\374\364\364\350\350\3109W\026\375\2153\375\344\321\303\223\243\243\007o\276\261\275\267\333\353\367sc\326\336\247$\204\332fV)\010\201D8%\022\021 TH\336%D\002\005\353\252\251\232V\303\377\317\206\300\234B\335\325\365r\376\331\243\207\207\237\177\326U\313aY6M\245\201\223H\364\255\363q\006H\'G\231\315m\221\373\266[\255V]\333\002\200\001%\210\253\331yY\226\343\203k\253\305\354\337\375\333\177\365\363\217?\372;\177\370\207_\375\352W\005Ek-\222\204\205\023\220\002\255\221E\261\304\215I+\245E\030\t8\302z\275\016>i\221\204\210\362\\\344/:WN\341\371\223\227\327\261B9\273\274\370\374\323\217/\316\316\266\247\343Rs]\255\200\275\357B\254\327\265\363M\353\\\022T\032\225\n!Xk9p]\327\232\324p8\314m\336\313\263\256k\222k\262\274\204\030\316\217\237}\360\376\217ze\276\267\267\247&SD\025\003{\037\224R\306\222\321\020\230H!3+\005\002@\010\336\303r\271b\346_8\314\375\227\333\025\003)\255\227\253\343\343\303\323\243\303\340\2732\317BC]\265n\326k\357}\325\266U\333u>\272$\002$\0101\306\262\354[m\234s\256\355\026\363\313\351h:\030\014\\\327>\375\374\311tk{{w\307X\375\321\317\336o\333\366\376\375\373\327\357\2775\036OM\2263\307\224\222A\"\002IIk\035\323sO\312\3204a\266\\\004\026\r/9\376_\331\371%7\332\264\325\311\311\361\351\321q\333\325\342\374\272\232\237\237\234,\026\363\343\243g\316\271\266\363.\245\010\212\005\203HJ\322\353\365\316N\216SJ\271\315\214\311$\246\313x^\327\353\335\335\375\004\262^\257\"\247r\320\357\017\006\317\236|\266\\\316\337\311zw\356\350\335\375\334\030\035#\023\201\010\204\020\2122\337x\360\215it]W\255\033f\326\035\026\000\202\020\221\"\001#0\202B\321 \212=Xk\265V\215_\263t\332`J.\236\375\364\331\373\177\331\254\252~\257w\274\272<\276\2748[\257\237\235/\026NU5\270\210\211u\010\201\214%R]\327B\335P\246\255-jD\342d5\026:\224\004\322\235\017\313\242\237\345\010\314m\'\320\370\270^.\216O\367\2467\006\271\235\354\000`H$\021\022\001$\255\204s\0358\005ff=:Z\207O/y\255\257\351\215\244\361\352\334\202\370\205SB\245\010\021SJ\220\2304\245\344\253\252:|\374\330\307\000\000\313\345r6\233-\026\263\325|\261^\257\2656\212P!$I\234\002!\n\021\244\010\t\330sH!\021\020&\311\224\301<\242p0!P\324\nY(\211WZ!i\255\037~\360\263\341hk0>\030\355\036d\032\252\n\214\205\311\300\272\344\0233\240\002T\336CU5\336{\021\255\257\202\204\315\001\006\001E=\377\207\020\024\"\206\3201sF\252sq1\273899\001!f\236\315f\253\371\242\255\233\030|i\215\213\001\242\223\030)%L\202\030\014\031!\000\322H\202\310\302I\221(A\202D\242R\364\022(*\2454\'\204\030} \305\314\325\351\311\017\377\323\3374\255\274\363\356\367\356\334\277\306\010u\223\240\257\2309\245$\244\210tS\371\263\213Y\343\275\220\321\233\243\352&\322\001\020 D!\020\020\021\255\024\00003\242\030\255\353\332/\347s\026L)\256V\253\345r\311)(\224<Se>=<>Q\2202L\240PY\320\032\263L\261\315\0002@\331\304V\010\254\265*\024Zd\003\254\001\214\"\243U\246\265\325F)D\024\t\376\370\351\347u\223\362r\262\177\343Z\231CS\207\365:\032+\302\212\001\200\314j\275>\273\234\373\310&Szs\024\003\374\205\370K\000A0\211\320\346\244I\204(\311\273\246ZYk\027\213\305l6K\336\027\326z\353c@\245 W\034\2240i\245T\010\210\210y\246\001\300E$BEj3\013\t\003Gq!+\213\\cat\231gy\236\347yN\200\314\2429\000\307f5{\374\311G\257|\364\352\355{\367\312\302\002D\021!\322\211Ud\\\254\232\313\371:\201\"$\2759\341\210\310\025\331\"\002BW\336\212\021\005Q\024\010G\337\265MS\257\225H\3234\"\322\353\365\024\021\002c\n\256\355zF\263\321J)k\255s\024c\324J\000\300\022(\245\264&E\210()\005N)\245\320\263\252\314l\277\260\275^\231\347\271Vv\023a\215\007\203\306\205\246^}\376\351\303\237\037\\\273v}wo\1772[\005\004TJ\271D\336\313\345r\275\250*Q:\010\350+\322A\000$\241\220lb\026ADN\214(D$\222\272\246\256\253US\255\261\252C\010y\236\023Jp~X\026\375L_\\\\h.{\006\262,+\313\3229\347\\\330\234\262\230\214R\312j\2435i\002\221\024S\2101Zk\007\375r4\350\345e\221e\205\000u]\027$\345\032\353\332\013\343j~\366\363\017\177\362\306\033\017vv\'\3219e\215V6\306T5q\261h\2336\200.\\J\232\204\001\030D\204@\030\023\300F\331\0321qR\010\212 %\366\256\355\232UtM\263\\z\357\025\221\357\332\325b\266;\035\r\'\243\3246\243Lw]\331/{\303\341\320{\337\272\216P\003\200g \"k\2241\332*\r\310\222bJi\276\\\014\212lP\024EYjkB\342\340\001!.f\027\204fw\2725\257\272O\036\376\354\375\037\377\365\255\333\327{Y\326\206\204V\247\310\336\305\266\365!1j\355c\322\314\214$D\317cOD\000\"!a\014!dy\026}P\nC\333>}\374\0309y\357\235s\206\220\243W\022\313\"\337\231\214\301w\004L\200Y\226Yk\233\246[.\227\314P\224e\033\222VW1Anp\330\037*\245\234\357&\303\036\013,\253E\364\335\326\316\236\204\3205\353\302\332\343\363\313\301p\014\022\021R\2574\207\317\036{W#\"\201\356\352.\263yS7\027\227s\006\3641&\021M\260\331\003\030\276H\207 \010\253\347\273\257\326\004)5\325j\275\\T\325\254i\352\340=#$\337\n\'\203`I2\215\271\312I\001\202\"\221A\221[\302\331b5;?\323Y\231\210\204#s\344\330\251\024{E\231\033\322Fk\255\025\241\000J\354\274\353$E\235Y`i\233\006\314R)\243\255Y-.N\236}~\373\336\003\2535&\3239\230\317W\325\272c\0064D \204$(_\270 \006\024\221\310\240\255\321Z\013$\255\224p\254V\313\256]s\360m\323\204\340\202kC\327\242\200Q\244\025\025F#\304\236\265\303\262\310\265\311\214\355\225\203^^ \243\210\244\224\0201#\r!U\213\345\362\362\242Y-5s\241\365\270\337\313\215\n]\327\256\227\31151\264\000\344\234[.\347\010)\267j\265\234=y\374\261\002\211\235\007\306\266\n\213\313ES5\000@\004\"\351\312\013]\035\225\021\t\t\004\t\325\346\010\344\275\327F\274\357V\313y\364N8z\3579&\220 \211\263L[B%l\021}L\212\250\314\013\006\214!%\300\335\335\375\321h\353\347\237=n\232:\370\316\"*\202L\2512\263\275\242\344\304\30217\231X\355|\013\034\205\203\357jD\245\010}\214)\005\002\021\366\227\027\347 \022\272`\n\364\255\257VuJ\311Z\353\t\021X\003\213\220|\261\002\210@\024\n\265m+\"\034\"+\364m\267\232\317\353\365*\270:\270NR`\340\\\253~Y(\302\340<!\346\326\004\347;l{\375a\221\367\231T\226\025E\257_N\'\307\207\207O\237<\251\026s\210\301\033\312\265\3522\233\345\206}\000\255s\255:\255\362L\245DM[\013\014\262<\227\344Bt>tY\236\265m\335\325\265\302!\n\266u\267^\254\203\367\250\2201\002$\r\300W\231A\021 \004\021\020\021A\357BnL\n 1um\275\256\226\256m\327\253\231sAR\3105\366\006\343\235\255q\226e\220\242\265\332\250|\261X5M\207d&\333\243\262(MVfE\376\225\267\33798\270~\375\340\332\362\362\262]/|\333H\362\256\353\212\334F\357\214Q\242H8jBBi\326\253\010e\006\204\210\336\373\252ZM\246\273\325zy\364\354\360\340\372\300{\277\230\317g\227s\337v\301`\324\214\000\372\352\020\261\311\313\246$\302\302\014\")%0f\275^/\375\362\364\360pvq\231\202\347\0249\372\024=\232b{:\276v\355\232\001\026\357{y\026B\030\016!\204\264\311\311ie\020T\327\372\344CQ\364\356\334}Mn\337b\347\3305]\263n\233\252^-]tZkFAf\205\250\0208\205.\272D\"\232\t\310\255\227y\321;==}\364\350\321\326\326mVY\014\\\344\371t:\255\225!fC\244\211H\020\004\204\231\031\204\0318\2410\007\347\220\371\364\364t5;<;\374\364\374\374<\305HD\033O\245I\215F\243\355\355m\010.4\315x\320\257\327\353\311d\213Hw>\2412\250\2152Z\"\353\314\226y\246\204\333\212\223tZ\353^o\220k\225\033\335\271FY\355R\254\332\206\025&\340\242\310\027\263\344R\2452]\350\254s]\3234a\335\035\035>\r!h\245\206\375\376\365\353\327\313\010\013\206\263\246v$:hA\001\337:\rj\322\353K\010\365z\316\276\315\242\353\332e\336\035]\\>Z\235\177\202<w~\036cH\353\265V\352\325\203\375i9\010\225\033\226\275\242\237\033SL\267\267<\213K\242\254b\235y\233\007e0\323\343\236m\333\226#\333b\240\265\255WK\347\033\255\373\322/L\021\024\010\306\270=-\275s\227\227\227%\257\206\205\253\3526\004\2148\354\025\243n\325eY\366\311\317\376\352\337\307\345\177\365\275\277\365\340\340\372@\273eG-\366\226\235\r\242t\2141\267Y\236\347\321\371\325j\351\353*u]/7\256\363\316um]\267m\233bDD\255,\021\245<\270\256\353\272N\020\362<7\306x/\000P\267\r\003%T\233\260Bkm\363\302d\271\325B\2001\006I\001\000\2141\212J\004\321\010 \311\020\206\020\264\326)F\255\225\010_~\362\251\367\276\r\211l\206\306\n\222B\342\"~\364\321GZ\233\275\033\257\352|\234\215\256\223\201\242\314\206\371P#+\022\024 \215\344C\273Z-\272j\3452\035}]-\317/g\347\253\345\242\353:\022\320\250\031 \201hc\212~\257(\212\004\322\270.t\016\000\214\261h\25429f\031\352\034\265\005m\020\245k\332<\317\212\"O\301\201\244\320\265M\3234u\225ie4\365\213\222\010\363,\243\"\3172\213\210\217\316\316\317\316/\353\2726Ei\313\036st\302]g8\306\207\017?\232/\353\353w\336\334\031ls\312\204\224\261\250\255\262\301s\362^!k\205\301Ug\'\237\237\304VIl\326\213\371\354\264^/9x\000D!\016\3304\315\366\366\366\326\326Nb>>>\345\340S\214e\321\357\367\3736/\212\001\364\213\"\313sP\3121;\027\244k\265V\004P\255\326\213\345\274Y\257\326\253E\265^A\212F\351\321\240\227eF\021\025E\216\002\004\270\273\265\375\364\331Q\n\021\0002c\\\010!:\327\326\375~\351\\\273\256\226\253\365L/.\330\206@e\033\242\006Q\034S\214\2219\266\325\374\344\370\351\263g\237br]=\367m\035\\\313)\000\200&\223\210\021ec9\200x>\273\\\317\027\275\242T\200\353\272Z\256+el9XO}\330\332\305\274\327\227\304\211c\256u\n\341r9??=\253\352\225F*\212^\231\027\222b\n\036Q\274s\"\002\314\314\334v\365`0(\362\\\033\262J\347\271\025\022\3278\037\232\246e\243\303z\235\035\035~>oc>\275\256\312iZ\314t\n@@VY\357\233\263\323\343O>\375\371\371\321g\223a~\362\3543N\0018jR\231\316\210\210\2304\330<\317#\247\305ze\024\366\372\375\353\327\256\021\240\3671\245\304\002\244t\010\241i[!\305\010)\261\260\370\340\243\017Z\3532\313\021\321\032\245\t\307\303\201\357\332\350\273\266\251\332\272\366\336K\n\321\371\030\330Z;\354\365\263\334\000\213\304\020\223\027\306\024\2022m\020X9\017\263\365\344\232\037l\271&\200\006F@m\024\243\261\004\3345\253\371\342\\\274v\276\226\340\202\367\310\250Hk\312\020\324\2464SUU\3234\333\223\2516f\265\256\353u\345\275\317\262\\\031\235\025=\345}S\257\231Y\031\rJ;\037c\214)%M\324&\271\2708]-\226>t\323\361\250_\026\333\343\321\246T\230\202K!\306\340\326\353.7v2\231hc\253z\265\\.\273\2561\226\362\334\372*\332\316Q\321%\323\211-\304\026>\221\326\332r\354\000\251,\313\351t\332\353\025\276mN\327\365xX\370$!8\327\004\016Ld\215.\224\322M\333h\255SJJ\251\363\363\363\307\217\037?\375\354sfVJmo\355^\277\371\312\365WnL\267w\207\3438\030\014rc\254\246\266n\227\313\345jU\035\037\037?~\374\370\370\370\320uMp\376\336\235[\337\370\352\333\373\273;)%L\321\207.\371\3205uf\365H\r\\J\253\345\372\362\362\334{W\346\232c~9_\333\262\352O\001\n,\333z\030\275\326\271\336$N\010\205\231Ed4\032\025E\361\350\341\307\353\245\215\276\213> +\005*\313t\236\347\243\321X.\353\266mg\263\331\326xR7\355\305\371eQ\024{\273\007!\004\"\252\252\352\364\370$\245\204(\275\"\267\nT\"\002l\252\372\370\370p6[Xk\367\366\016\\\333\314.\317?\375\344q\256\225\325o\227\231&\020di\353J)%\222\274\017\255\367m]\371\256K\t\242\216\'Gk\233\203FH!\222\345\272Zuu\265{cG\207\340\264\265\321\257I\245\355\355\355^o\340\023\367\373\303\305\342\002\242\027fC&!\247\324p\202\256\353bt\233\2525\221\332\337\335\353\275\333W\200y\236[k\211hS\347\323\231-\214\205\030\202k\333\312\271\256A\304\321`X\226}\000\010!\270\266\323\232|\263\336\231L\246\323\251D\347\233\365j1_\314\347\2552e^\032\233\363rU\366\212\203l\277\255\253uU\365z\320\266\260^\327\343|\334\317\363\242\350)\245PD\207\344\363\"\357\332\250\210\'\323\355\351\366~f{Y1\244\345*\202\210\204\310\242\021\200\242\217\016=Q\212\326ZH\234B\334\336\336\276s\347\236U\266i\232\242(\214\322\200\354\234\353\272f\223\005P\010\241\352RHyf\364\326\2261\2316YJ\311\371\326(\"N Qq\020$\000p]\027]\007\271b\211)&\037\272\350\203H$\002\253\001\021\265\026m\263\262,\313\242\227\031\213B)\262&-\221#(` @u\260\177\343\346\255{\263\363\213\361d\267\255V]S\tGP\244\214\311\212,\317\255\324\016\001\202\367\3169\022\312Mn\214\t\316Kb2D\210^\200c\022\340H\024\020\\\347H\2512\313\223 \220BR\000`t\006\222\262\322\372\246j\333\326\240hR\nD\033\2258\264\213.!\3466\253U{y6\217)\225\245\2111\356\357n\r\267\366L9aeQ\264\010\006\027u\226\353\256k\2151 q\265\252\267w\016\336z\353\235\367\177\362S\253\255$\212\221%\005c0\313M\226ik\225NE\327y\016Q!2sS\325y^\022mr\016H\202\220\230@\010H\213@\014\034\202\265\326fyd\360113#(\245\2146yf!\006\2679\311K\022\021I\334\2645eYQ\016A\233\316;k-%_fe9)w\367\257\225\203\311\242M-Car\215\266\256\235\026\222\304.\313\013\tXU\355\316t\362\352\3357n\337\272\177~\372\214cdf\204`,Z\275\251\336\306\314Z\337\005\205*\267\2315\006\000\264\326\375\262g\255%\201\266\253A\222\026\324\2064H\352\274\2656\313\262\274(\030(\027@\322\250\265Fj\232\032\222\337\324iRS\247\030\t\021\022\023\321\260\337\023\245\316g\027\313\345z8\034\366\372En\354x<.\313~\343\245m\235\230\322\230\234\310\272.h\237:2\n\010\022\010*\223\222\014\372\343w\177\343\333\377\361\007\377\001XR\010\211[c\2004\0231\222P@\243\265Q\032\205\024\350\334\026E\226km\013k\275\367\321\207\344\203V\252W\024\212\240\353\032\006\005)E\037\320\330</\362\242\247\254\005\200\224\342r\266\342\224\224R]\010\222R\236g\306\2522S\000\320\266m]\3271\372AQ\014{\303,3J\251\266qm\020\205Z\224q\255g\355\225)\310{o\255\025\021a(\212\322\207\004@o\275\371\225~\177hm\226D\272\256k\332\326{\317\0227\221\317\246\270\037B\2101n\372\3169f\216!8\3476I\253\321`8\032\215\306\203a\360\276\252\252\371\342r\271\\v]\227RJ)9\347z\275\336\013\330\301\306\203[k\2151\"\262^\257c\364\373\373\3737o\336\264\326\256\327\353\246i\346\227\213\305b\001\214\303\341Xk\275\\.\253\252\312\363\234z`\260\213\022@\031\033\225\361&\343\361\310\354\035\374\341?\376\307\327\337\372\n\365\306\223\335\033\327\366of\234\371\231\247\212O[X&5\334\277\236\225=M\334,O\362T\351\356\334\317\236\036}\372\223\313\243G\212R1(\365h \375~\223\227\375I^\2163\037\033\220v5?9y\372H\307\026]\265\2728,5d\304\004\251?\310QA\227\334hk<\226\3632]\356\224J\234\337\236\354\257\253\210\224\317\347\313\252\276\024X\027=\016\322\\4\265\014\366\365\316\033O\333\361U}\340E\231ID\000\201Av\367\016~\343\335\337t\325\374\375\367\376\272\232/\207\205\261Yqvq\271n\301(\232\315f\373\223\t(\312\363,\2010\363g\207\207\375~?/\212\363\371\342b\261\274\021B\321\357\237\235\235\235=\375d2\231\364\312\301d2Y\257\353\331\345\334\332ck\324\246^\010\000(\200(\306\030\341\314\020^\273\375\332\354\303\217\205\314dk\372\301\307\217\252\246\336\335\337I\2611\232\362\242GZ!\350\336`HY\026\005D\351_Q\245\024F\004h[w\363\366\355o\374\3327\027\027g\207\237=\322\271\001\255y\276\024\214\201\323j\275F\255B\010\\d\353\365\332\267\315\316\316\016\"\212\322c\241\263\331\374\263\307O\212\376\200\021\366\366\366@\350*\237g\214\265\266i*G\224i\315)\244\350R\nD\244\265\"\314\331\332\213\313\305\315\327\336Z\265r\276\252MV\2742\332j\232\306{\257\264\026\302\326\007\257\245\034\214So\320&IJk\272\202\034mp\n_\244\267\262\2424\224\035\334\270}\373\356\203\305b\261^\3152\245\266\257\337\324\213\205o\333$\254\264\356\274k:\265n[\t\276\2656F\036M\'\373\007\327G\333\273\253\266\025\240$8,\257\200=\240t?/\230yv1\253W\253\321\260O(\n\004A\210H[\313\t\2309\330!\243\236\315\026\207g\263\355\235\003c\314\342\350\202S2\021b\222\350\203h5\030O]>\254\275&\233\351\347\365\031`\331T\006\200\021\220\311\307\244rs\360\312\255o}\357wF\223\361G?\373\321\323\307\217W\213\231V\272\r\241\352\\\000\326\212\242\200\017\036\022_\333\333e\206\304pqq\221Pm\355]\033\216&u\327\006w\231\347\271\357\302j\265\222\2304\251\301\240\277=\035\035={j4\025\2315\326\020RJ*\005\2121n\337~\363/\377\372o\262b\334\245\313\307O\017c\350v\247\003\227: \025DD\231b8\036\357\354\325f{]\251\250J\275\261\302\rniS\034\020@\006n\252\006\261(\213\374\346\253\257\357\354\356\337\177\343\255\237\374\350o~\366\376\373\207\237\376lQ\325\306\352U\323N\367w3\243\025Hr\335\351\351\371\351\351ib*\006\003@\363\354\3442/J\223\025\"\313\315V\275^\255F\375\201&5\2738\353\025\245\321d\214\311\262,\317-\021\204\350\034!\023=[\307w\277\367\373\333[\373?\376\350\177[.\226\205\205\266\363M\323f\275\314%\350\365\006\333\327_\231\\;P\260\265\324\262rx\265\210\001\021\204\001\3619\004\213\266\367v\235k/\346\313\024C/\3237\357\336?\270q\353\333\337\373\335\177\375/\377\367UUq\014\363uu\357\366-R\330\263\272\253\340\361\243O\036~\370\363\375\203\353\177\367\267\177\347\326\335\327\217\216\317.\227\253</\223,\200\331j\003\302\205\315\236<\376\364\203\367O\372eq\377\365{\326Z\233Y\225\351+\315Ga\306`\206\277\377\367\377Qa\354\377\361\317\377E]u\223\311`>;&HA &9\330\332y\365\376\203\341+\257-y\304\303H\027\225\3762\\P\333\371\340\203\311z\323i\021\202\233\257\226(2\334\336\377[\377\315\037|\376\364\331\341\347\237\327]\227\220\002\244\322\346\216\352\327\337xpvvq|v\372\263\017?\330\336\177\345\355\257}\235\362\022\030\240L\365\371\351\374\342\362\311g\237>|\370\360\307?|\357\364\360\331o\177\357\267\212\242PFkc\230(\t{$\207:\240|\353w\377N0En\355[_\377\365\237\177\364p]5\200\2527\034\203\266`\363\361\316\265W\356\276>z\345\325%\017p\354\331^\\U\352\177\261Z\317\000\230R\"e\004y]\267\"Ie\031\2624.m\037\274\362?\374O\377\354\177\375\343\377\371rY\005\201\014M\326/\253\365R@\356\277\361\306\247\237>\371\313\277\374\353O\236\034\275~\377\315\376p\332\264\335r}\350\234[\\\234\237\034\0376\253\325t8x\347\235w\266\266\266:\357v\'c!\265n\273\004\252MJ\017\306wn\336\336\376\346wfg\247\273;[w_\273o\263b\276\\\"\270bXF\264o\276\371\325_\377\316o]\177\365\325\206L\216j\177\257\000\332{\256\201Me\030\220@\204\201Qh\203\206\333T\220E\201\020S\022\221\305\322\225y\377\366\275\373\237|\370\323\223\363\313\233{\333\353\266\035M\'\315\272\332\332\331\026$stZ7\335\243G\217\206\343m\233\345\2720\322y2v{{\333\356\356n\215\206;\333[\306\030\001\352BDkT\331s>\255\352\2605\235\354\276z\237\2655Y\357\364\354\342\301\233o\375\217\377\364\237r[\035={\374\177\375\207?\351\027\203_\373\315\357\276\376\366\327j\000A.r\350\022\2246j \002\270Bz\221 # \261\022J(\270\301\020~\321\024\200\224\203\251\325\364;\277\373\007E\226\317W\353\235i\034a\316\010.xk\314\356\301~o0^\255[$3\331\332\233n\355\234\\<U\200\212\300NG\323\341p\334\357\021H\344$\204L\006I;\246F\320\216\266\'7\356\214\256\275z\324\2666\317f\317\316\356\334\271\363\225{\267\331\325O\237>\371\350\311g\017\276\376\265o|\373\273;;7V\347\3476#\225\201\251\334\366\340\371\"\336\324g\020\005\004\010\220\221\325U\332\232\340\n\340w\325\006\343\335\325r\366\346\333\337\330\231\214\377\374O\377\317\266\013y\321[\317\316\372\203\201wNR\352\017{\275\341$2\032\233\001\362\366\366v\257\310\\\327\267$\203\242\310\024u]\343;\257\262\\\231\334\201\2325\035\024\375\033\367\356_\273\363\232\323eh\273^\236\345\205\3152C)\2102\267\357\275\366\007\177\367\357}\345\233\357\366\'[\363\256A\255\362L\307\350r\252\363\242\304\017?9GD\000~\001\231\026\021\006\331`\010\001\210\237\227\236\222\260\210\244H\230\202\201\2705\314?\374\321_\374\374\307\177ucw\254\305\217\373e\265\230;\027\212\274o\263^H\230\030\2252y\317p\36418\t\035\246D\234\230cd\310\207\243\000j\235\260C;\276q\373\265\267\276>\334=h|@\315\340\275I\321J\352\226K\024\351\r\372\027M\265s\373\346\311j\335\212\350\262`\346\024\242&\244\r(\360\n\313\373\022$\005\005\200\005XP\030%\241\244\315C\024\350<O\267\017f\253j\265n\036\274\365\326xk\373\344\364\334X\333\266-\020\226e\236\227\271\265\272,\363\311d\264\273\273\035B@T\0338xJ)\244dm>\034\217\006\243\341\272\365Q\324\365\333wo\336}\240\006\023\007&\351\242\227\231\246Z\n{\016>\267v8\034V]\227\365\206\027\363\272\362Q\3479\"\324\325\022B]\250@~\246y\203\351\333\374l\020\334p\005\335\306\r\212\031@\000\010\230\205\005\241(\307!r\1770\351B=*\262\327\357?x\224\332\272\256\255\202L)\2555slCL\242\214M)m\312\005\014\000D\244\265&P\332Zc\262\326\005Rz\347\340\340\346k\257\353\341N\213&\002D\304\344\032\222\310AbL\245\311L\226\245\266\255\352f\031\243\035\217|H\256\3534\261\202$\255\353) \331\340S_j$\200,x\305\313fw\27620d!R\037}\374h2\236\226E\377\370\364\344\336\275\327\337|\363\315\305j\351\234\013!\244\224\000\240(\212\351t:\231Lz\275\336&\203\215\210J\251,\313\262,CD\027\303|\276\354\r\007\267^\2753\231n\013\022\220V\032<C\333\325\223\321 \317,\000t]W\327\26559\240\022  uvy\261\\.\373\375R\0237\365|\330\317\360\203Gg_\034\2447\346\204p\265\030\360E\341\025\322\363\316J\005c2\357}F\326 P\350\272\213\223\017\376\352\317\260\231\367\211\307\275l8\350im\023\000\231\"/\207\243\352\031#\255\2728wQ\362\201\032NW\001\216\346\325\356\355;\327o\277\266\177\375&i\343]DD\255\225\010\310\352s\245\024\001J\210\301y`\266\326Zk/\347s\262\272\nn\2605Y\267M9\031\2225\341\005z\375\377;\360\014\001P@1\021\241F4J\213\265e\226+\350a\267\256\227+\356\272\242\327\'kc\027\227\3135K\335\370@Y\321\033Lf]8\374\354\211\352O\256\335|\365\225\333w\373\323\351&\232K)\031\235m\306\327Z{\357\203\363\03203V\0231\363&m\263\250V\363\3652\201\2502\367\336\023\210\335\000J\276\300\372?\027\377/\265\227\221,\030A)\000@K\312\020\346`\310d\375\274\2708}\246c\245\243\213\rr\214\275\301X\031+B-\252@\330:\356\3342e\305\301\315;{\267\357n]\277\245\213\001\330,&\336X\035)\000\341\030\202\332\2048\",\022c\004\242\030c\010!23\2631\246\353\272\351d\244\224n\332\256i\032\375\237S\177eK\370+\250\007\000\214\254\255\"\205\2716\226\320F\004 \005\030;\327\323J+\323T\253\331\331e\004D\235\001\251b:\252\233\256\366\261\267\265\367\332\333\257\276\366\325wF;\327\274Rmd\n\211\201P)\245\221\010R\010>4:%c\214Q\312w\256\256\353\r\334\317Z\353cD\304z\265\376\351\303\017\007\323\361\265\3337{\303\201\3152\375\245\324\177\211M\241\220B\312\264\315maD0&I 1eJgF\014 +\323H\335\265>J\223D}<[u1\335\270}\347\253\337\372\356\033\357\374:\333|\351\242\030\352b\322\300\306(\"@@\341(\034\010\000X\030\031\230c\214\314\314\"\032@\020#\'\001\331x\205\263\26334\272\\\366\246;\333\372ey\277\350\277P\313\313\342\337\274\243\2656&\333\370DL\351\371~\007&\313\\\267\014\311\221\321\323\3514\367\251\013!2\034;\375\326\033o|\363;\277\365\332\233o\353\301\240r\014\030@Ym\205P\013\n\010\307\024\230\031$\031\205\342\245\353\272\350=\n\024E\241\265\336\200\3156N\345\372\365\353\305\260\3777?\372a\221\345\363\313\031]\325X\377\337\250\177\361\016\000\330\2740\231%\245\231H\020\230Hi\255\262\274\030\014\223\322m\210m\364k\347.\327\313y\323x\200\257\177\373\267\177\353o\377\341\335\257\375\332*\302\343\243\213\225\017\220\365\232\230Pg@\270\331\205\024\002H\222\030bh\233\246K>\201\2201YV\224\326\346J\031\000\032\217\247E\336k[wr|\366\343\367~\254@\225y/\372\364\013A\375\227Q\377\013&\224\031\321\232\231\003JBT\n93\252\310=@2Z\365J\243\250\253+\010i2\232^\273y\353\355\377\366\357\231,\357\022\261\315r]\260\326\216\031\215\r)Jb\215l\2642\312$\211]\353\202\353BU\227e\331\357\367\215\3261\306M\306\211\264b\346\361x\\\373n\275Z\035\037\037_\277v\255n\232\307\217\037\353\r\224\004\010q\2033\000`\346\264\271\365\262y. \"\364\274\245\334\262\241\020\022\020!\246HHy\236M\306\351\334\370\206\312bxrz\"\010_\375\316w^y\365\356\265Wn>\365iP(A\004\245]\027z\2719:\233\355\354M%AH\236\231E\000DRp\276k\273\246\276u\375z\3234Z\353\315\255\264h\014\"VU\325\353\367\337{\357\275\177\365o\376\265.\2627\037\274\361\360\203\017\275\367\337\377\263\037\020\020\376\222\214\3459\356\343\305sD\004E\250\025\031-\212@+QZ\024\261&\321\006\254\225,\313\307\223*\361\334\371\255Wn~\373o\377\3767\377\353\337\335}\365^\247mU\255\216\217\217\227\353\225\265v<\3565\215\333\331\231\266\265\027\021\245Ik\215\002\033\307\257\220\372\275^\010\251_\0162\223#\250\371l\tBZ\331A\177t~~\371\361\307\237\214F\223\017\177\372\241\325\331\361\341\311\336\316\376\301\366>\275|\177\340e\313Auuc\215\021\204\256r\200\326Z2\032\024\000\021\023\210\322`\014f\031f\305\274m\213\361\326\215\273\367\277\362\033\277\371\372\327\2771\330;\220\262\237lo\330/s\215\205Q\311u\307\207G\177\361\037\377\3147m\231[\341\244\221\214\"\002\t\336\'\037\214\322\243\376\200\000SJ\233\244\345f\005{\357]\360\337\377\376\367\027\353\325\321\321\321\357\375\336\357\271\246\375\355o}\347G\357\375\360\323G\237<\017\352\001\322s\353\337\310\376e\3104\300\006y\250\255\265\016P\275\274$H!\031\245\263\305\272{p\357\336[\367\357\r\307\243\244\262g\347\313y\335FQ\243,\013\034\333\325\374\374\374\364\311\341\311G?\371\211\357\334\267\276\363\355\242(\020E\021\242\000\t\013\242U\332Z\233Y3\237\317\007\203A\333\266{{{\033\325-\317\317\307\343\361\303G\037\337\270q\343\366\255[1\204?\375\323?\215\336\377\367\377\344\277\323\033\374\306/\271KB\272\302ul& \332\224\217\224\326\304L\014\304BW\327\227X\004E\320d\371+7_\035M\367c\214\t)\242u)\234\315\346w\037\334\356[\335\305\324\357\367vw\266\372\375\362?\375\370\247\203\"gN\010,\014\004`\264V\332\020\221$\376\223\377\373O\206\303\341\273\357\276\373\336{\357}\355k_;=?\333\335\335\375\363?\377\363\255\335\235\203\203\203[\267n\375\340\007?\270w\357\336\355\333\267\277\373\335\357\346y\2567\227\376\340E$\360\322U8\276Jy=\317\273 \002\000\305\210J+a\305J\0040\201D\211\201!\321d\274\223\347\275\305r\315\240\215\315\206\343\322\261^\236\037+k\0204G\227\365\206\205\306\223\247O\210#2\\\t\010A#%\245PD\242<y\374\344\265\373\257\237\234\234\376\361\037\377/\357~\3637\356\336\275\373G\177\364Ge\177p\353\346\355\177\373o\376\335O~\374\323\177\370\017\376\301\267\276\365-c\314b\261@\206\377\007H\265Mi\267\346\222\214\000\000\000\000IEND\256B`\202"
          }
        }
      }
      feature {
        key: "image_id"
        value {
          bytes_list {
            value: "193353.jpg"
          }
        }
      }
    }



#### Load the keras models


```python
from tensorflow.keras.models import load_model

model1 = load_model('smile-model.hdf5')
```

    
    User settings:
    
       KMP_AFFINITY=granularity=fine,verbose,compact,1,0
       KMP_BLOCKTIME=0
       KMP_SETTINGS=1
       OMP_NUM_THREADS=2
    
    Effective settings:
    
       KMP_ABORT_DELAY=0
       KMP_ADAPTIVE_LOCK_PROPS='1,1024'
       KMP_ALIGN_ALLOC=64
       KMP_ALL_THREADPRIVATE=128
       KMP_ATOMIC_MODE=2
       KMP_BLOCKTIME=0
       KMP_CPUINFO_FILE: value is not defined
       KMP_DETERMINISTIC_REDUCTION=false
       KMP_DEVICE_THREAD_LIMIT=2147483647
       KMP_DISP_HAND_THREAD=false
       KMP_DISP_NUM_BUFFERS=7
       KMP_DUPLICATE_LIB_OK=false
       KMP_FORCE_REDUCTION: value is not defined
       KMP_FOREIGN_THREADS_THREADPRIVATE=true
       KMP_FORKJOIN_BARRIER='2,2'
       KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
       KMP_FORKJOIN_FRAMES=true
       KMP_FORKJOIN_FRAMES_MODE=3
       KMP_GTID_MODE=3
       KMP_HANDLE_SIGNALS=false
       KMP_HOT_TEAMS_MAX_LEVEL=1
       KMP_HOT_TEAMS_MODE=0
       KMP_INIT_AT_FORK=true
       KMP_ITT_PREPARE_DELAY=0
       KMP_LIBRARY=throughput
       KMP_LOCK_KIND=queuing
       KMP_MALLOC_POOL_INCR=1M
       KMP_MWAIT_HINTS=0
       KMP_NUM_LOCKS_IN_BLOCK=1
       KMP_PLAIN_BARRIER='2,2'
       KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
       KMP_REDUCTION_BARRIER='1,1'
       KMP_REDUCTION_BARRIER_PATTERN='hyper,hyper'
       KMP_SCHEDULE='static,balanced;guided,iterative'
       KMP_SETTINGS=true
       KMP_SPIN_BACKOFF_PARAMS='4096,100'
       KMP_STACKOFFSET=64
       KMP_STACKPAD=0
       KMP_STACKSIZE=8M
       KMP_STORAGE_MAP=false
       KMP_TASKING=2
       KMP_TASKLOOP_MIN_TASKS=0
       KMP_TASK_STEALING_CONSTRAINT=1
       KMP_TEAMS_THREAD_LIMIT=2
       KMP_TOPOLOGY_METHOD=all
       KMP_USER_LEVEL_MWAIT=false
       KMP_USE_YIELD=1
       KMP_VERSION=false
       KMP_WARNINGS=true
       OMP_AFFINITY_FORMAT='OMP: pid %P tid %i thread %n bound to OS proc set {%A}'
       OMP_ALLOCATOR=omp_default_mem_alloc
       OMP_CANCELLATION=false
       OMP_DEBUG=disabled
       OMP_DEFAULT_DEVICE=0
       OMP_DISPLAY_AFFINITY=false
       OMP_DISPLAY_ENV=false
       OMP_DYNAMIC=false
       OMP_MAX_ACTIVE_LEVELS=2147483647
       OMP_MAX_TASK_PRIORITY=0
       OMP_NESTED=false
       OMP_NUM_THREADS='2'
       OMP_PLACES: value is not defined
       OMP_PROC_BIND='intel'
       OMP_SCHEDULE='static'
       OMP_STACKSIZE=8M
       OMP_TARGET_OFFLOAD=DEFAULT
       OMP_THREAD_LIMIT=2147483647
       OMP_TOOL=enabled
       OMP_TOOL_LIBRARIES: value is not defined
       OMP_WAIT_POLICY=PASSIVE
       KMP_AFFINITY='verbose,warnings,respect,granularity=fine,compact,1,0'
    
    2021-12-11 03:32:39.041066: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
    2021-12-11 03:32:39.041714: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55645dc0cc60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
    2021-12-11 03:32:39.041743: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
    2021-12-11 03:32:39.041862: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


#### Define the custom predict function for WIT


```python
# This function extracts 'image/encoded' field, which is a reserved key for the 
# feature that contains encoded image byte list. We read this feature into 
# BytesIO and decode it back to an image using PIL.
# The model expects an array of images that are floats in range 0.0 to 1.0 and 
# outputs a numpy array of (n_samples, n_labels)
def custom_predict(examples_to_infer):
    def load_byte_img(im_bytes):
        buf = BytesIO(im_bytes)
        return np.array(Image.open(buf), dtype=np.float64) / 255.

    ims = [load_byte_img(ex.features.feature['image/encoded'].bytes_list.value[0]) 
            for ex in examples_to_infer]
    preds = model1.predict(np.array(ims))
    return preds
```

## Note that this particular model only uses images as input. Therefore, partial dependence plots are flat for all features. These features are provided for slicing and analysis purposes.




#### Invoke What-If Tool for the data and model {display-mode: "form"}


```python
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder, display

num_datapoints = 250 
tool_height_in_px = 700

# Decode an image from tf.example bytestring
def decode_image(ex):
    im_bytes = ex.features.feature['image/encoded'].bytes_list.value[0]
    im = Image.open(BytesIO(im_bytes))
    return im

# Define the custom distance function that compares the average color of images
def image_mean_distance(ex, exs, params):
    selected_im = decode_image(ex)
    mean_color = np.mean(selected_im, axis=(0,1))
    image_distances = [np.linalg.norm(mean_color - np.mean(decode_image(e), axis=(0,1))) for e in exs]
    return image_distances

# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(examples[:num_datapoints]).set_custom_predict_fn(
    custom_predict).set_custom_distance_fn(image_mean_distance)

wv = WitWidget(config_builder, height=tool_height_in_px)
display(wv)
```

    2021-12-11 03:37:35.851215: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
    OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
    OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
    OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0,1
    OMP: Info #156: KMP_AFFINITY: 2 available OS procs
    OMP: Info #157: KMP_AFFINITY: Uniform topology
    OMP: Info #179: KMP_AFFINITY: 1 packages x 1 cores/pkg x 2 threads/core (1 total cores)
    OMP: Info #214: KMP_AFFINITY: OS proc to physical thread map:
    OMP: Info #171: KMP_AFFINITY: OS proc 0 maps to package 0 thread 0 
    OMP: Info #171: KMP_AFFINITY: OS proc 1 maps to package 0 thread 1 
    OMP: Info #250: KMP_AFFINITY: pid 2351 tid 4097 thread 0 bound to OS proc set 0
    OMP: Info #250: KMP_AFFINITY: pid 2351 tid 4097 thread 1 bound to OS proc set 1
    OMP: Info #250: KMP_AFFINITY: pid 2351 tid 4099 thread 2 bound to OS proc set 0
    OMP: Info #250: KMP_AFFINITY: pid 2351 tid 4096 thread 3 bound to OS proc set 1
    OMP: Info #250: KMP_AFFINITY: pid 2351 tid 4100 thread 4 bound to OS proc set 0



<style>.container { width:100% !important; }</style>



    WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'are_sequence_examples': False, 'inferenc…


#### Exploration ideas

- In the "Performance" tab, set the ground truth feature to "Smiling". You can set a scatter axis or binning option to be inference correct and analyze how it varies across other features (i.e. you can make a scatter plot of Young vs inference correct).
- Choose an image and click on "Show nearest counterfactual datapoint", this will find another example that is closest to the selected image in terms of average color value, but has a different prediction (if selected image is predicted to be "smiling" the counterfactual one will have "not smiling" prediction).
- Define your own custom distance function and set it by calling set_custom_distance_fn on config_builder and explore the counterfactuals. You can even load another neural network to compute distances!
- You can slice by any one of the features and analyze the confusion matrix and accuracy for each group.
- In the "Datapoint Editor" tab, you can upload your own image or download and modify one of the images to see how it affects the inference score.

